In [42]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name, generate_aliases
from agile.utils.dataframes import modify_and_sort_columns, clean_and_verify_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh
import random

import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
df = pd.read_csv('data/weeklong_gh.csv')
df['datetime'] = pd.to_datetime(df['datetime'],errors='coerce')
df = modify_and_sort_columns(df)
aliases = generate_aliases(df)
df['days'] = df['datetime'].dt.day

color_map = {
    8: 'green',
    9: 'blue',
    10: 'red',
    11: 'purple',
    12: 'orange',
    13: 'yellow',
    14: 'darkgreen',
    15: 'beige'
}

df['color'] = df['days'].map(color_map)
all_adids = df['advertiser_id'].unique().tolist()

In [61]:
def data_map(container=None, data=None, lois=None):
    """
    Display a map in a Streamlit container.

    Parameters:
        container: Streamlit container to display map
        data: Pandas DataFrame of data points to display on map
        lois: Pandas DataFrame of locations of interest to display on map
    """
    if data is None and lois is None:
        return

    # Determine size of data to be displayed
    data_size = len(lois) if data is None else len(data)

    if data_size > 0:
        # Get coordinates of first data point
        first_point = lois.iloc[0] if data is None else data.iloc[0]
        lat, long = first_point.latitude, first_point.longitude
        
        # Create the map
        m = folium.Map(location=[lat, long], zoom_start=10, width=500, height=500)

        # Add data points to map
        if data is not None:
            data.apply(lambda row: folium.Marker(
                location=[row.latitude, row.longitude],
                popup=f"Location: {row.latitude}, {row.longitude}"
                        + (f" Node Name: {row.name}" if 'name' in row else "")
                        + (f" Timestamp: {row.datetime}" if 'datetime' in row else "")
                        + (f" Advertiser ID: {row.advertiser_id}" if 'advertiser_id' in row else ""),
                icon=folium.Icon(color=row.color)
            ).add_to(m), axis=1)

        # Add locations of interest to map
        if lois is not None:
            lois.apply(lambda row: folium.CircleMarker(
                location=[row.latitude, row.longitude],
                radius=30,
                popup="LOI"
            ).add_to(m), axis=1)

        if container != None:
            with container:
                # Display the map in a container, if there is one
                folium_static(m, width=725)
        else:
            return m
        
    else:
        if container != None: 
            container.write("No Data Points Available")
    
    return None

In [34]:
history = []

## Run the cell below repeatedly to explore ADIDs on the map

In [33]:
adid = random.choice(all_adids)
adid_df = df[df['advertiser_id'] == adid]
print(adid, f'Total Points: {len(adid_df)}', sep='\n')
history.append(adid)
data_map(data=adid_df)

536d6d47-5ff7-2166-9437-567259e7607b
Total Points: 523


## Run the cell below after inputting a lat/long pair to explore an area more

In [62]:
lat = '46.241837'
long = '-119.1010492'

ql_df = query_location(lat=lat, long=long, radius='30', df=df)
ql_unique = ql_df['advertiser_id'].unique().tolist()

available_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
color_vals = []
for id in ql_unique:
    color = random.choice(available_colors)
    color_vals.append(color)
    available_colors.remove(color)
    
color_palette = dict(zip(ql_unique, color_vals))
ql_df['color'] = ql_df['advertiser_id'].map(color_palette)
data_map(data=ql_df)

In [63]:
ql_df.sort_values(by='datetime')
    

,advertiser_id,advertiser_id_alias,Unnamed: 0.1,datetime,latitude,longitude,geohash,days,color
6455288,07049108-8a38-589c-2e50-7ce9f4033a96,None,6455288,2018-04-08 08:20:32,46.241630,-119.101260,c25r1f1152,8,orange
6623836,b4814478-4323-ad74-8cf7-1d714bf8d951,None,6623836,2018-04-08 10:08:35,46.241720,-119.101033,c25r1f138c,8,black
6626724,b4814478-4323-ad74-8cf7-1d714bf8d951,None,6626724,2018-04-08 10:10:07,46.241608,-119.100990,c25r1f12cu,8,black
6634025,b4814478-4323-ad74-8cf7-1d714bf8d951,None,6634025,2018-04-08 10:13:59,46.241600,-119.100989,c25r1f12cg,8,black
6636976,b4814478-4323-ad74-8cf7-1d714bf8d951,None,6636976,2018-04-08 10:15:31,46.241582,-119.101000,c25r1f129x,8,black
...,...,...,...,...,...,...,...,...,...
970876,3c5a034d-4b97-1543-9825-998416af4641,None,970876,2018-04-15 18:42:13,46.241690,-119.101015,c25r1f1335,15,purple
973687,3c5a034d-4b97-1543-9825-998416af4641,None,973687,2018-04-15 18:43:30,46.241600,-119.101000,c25r1f12ce,15,purple
974070,3c5a034d-4b97-1543-9825-998416af4641,None,974070,2018-04-15 18:43:41,46.241700,-119.101100,c25r1f11rj,15,purple
978820,3c5a034d-4b97-1543-9825-998416af4641,None,978820,2018-04-15 18:45:48,46.241600,-119.100999,c25r1f12ce,15,purple
